# 월간 데이콘 생체 광학 데이터 분석 AI 경진대회

다변량회귀 </p>
hyperopt(https://teddylee777.github.io/machine-learning/hyper-opt/) </p>
“device” : “gpu”

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error

In [2]:
os.chdir('./Data/생체광학')
print(os.getcwd())

c:\Users\movem\Documents\Python Scripts\Data\생체광학


In [3]:
def shownull(df):
    global missval_name
    print(f"데이터셋 차원 확인 \n {df.shape}")
    missval = df.isnull().sum()[df.isnull().sum()!=0]
    print(f"결측값 있는 변수 확인 \n {missval}")
    missval_name = missval.index # <- 결측치가 존재하는 열이 저장된 변수임

def RMSE(t, p) :
    return np.sqrt(mean_squared_error(t, p))

# 데이터 전처리

In [4]:
train = pd.read_csv('./train.csv', index_col="id")
test = pd.read_csv('./test.csv', index_col="id")
y_test = pd.read_csv('./sample_submission.csv', index_col="id")

In [5]:
print(f'--독립변수(Y) : {list(test.columns)}')
print(f'--종속변수(Y) : {list(y_test.columns)}')

--독립변수(Y) : ['rho', '650_src', '660_src', '670_src', '680_src', '690_src', '700_src', '710_src', '720_src', '730_src', '740_src', '750_src', '760_src', '770_src', '780_src', '790_src', '800_src', '810_src', '820_src', '830_src', '840_src', '850_src', '860_src', '870_src', '880_src', '890_src', '900_src', '910_src', '920_src', '930_src', '940_src', '950_src', '960_src', '970_src', '980_src', '990_src', '650_dst', '660_dst', '670_dst', '680_dst', '690_dst', '700_dst', '710_dst', '720_dst', '730_dst', '740_dst', '750_dst', '760_dst', '770_dst', '780_dst', '790_dst', '800_dst', '810_dst', '820_dst', '830_dst', '840_dst', '850_dst', '860_dst', '870_dst', '880_dst', '890_dst', '900_dst', '910_dst', '920_dst', '930_dst', '940_dst', '950_dst', '960_dst', '970_dst', '980_dst', '990_dst']
--종속변수(Y) : ['hhb', 'hbo2', 'ca', 'na']


## 결측치 대체

In [6]:
shownull(train)

데이터셋 차원 확인 
 (10000, 75)
결측값 있는 변수 확인 
 650_dst    1948
660_dst    1988
670_dst    1931
680_dst    1975
690_dst    2003
700_dst    1966
710_dst    1946
720_dst    2046
730_dst    2006
740_dst    1993
750_dst    2071
760_dst    2050
770_dst    1994
780_dst    1945
790_dst    2059
800_dst    2026
810_dst    1990
820_dst    2091
830_dst    1965
840_dst    2045
850_dst    2014
860_dst    2010
870_dst    2020
880_dst    1968
890_dst    2059
900_dst    1999
910_dst    2030
920_dst    2038
930_dst    1978
940_dst    2018
950_dst    2013
960_dst    2008
970_dst    2069
980_dst    1919
990_dst    1987
dtype: int64


In [7]:
X_train0 = train.drop(list(y_test.columns), axis=1)
y_train = train[list(y_test.columns)]

In [8]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [9]:
X_train_ipt = X_train0.copy()
imputer = IterativeImputer(random_state=2023)
X_train_ipt[missval_name] = imputer.fit_transform(X_train_ipt[missval_name])

print("\n변환 후 결측값 :\n", X_train_ipt[missval_name].isnull().sum())


변환 후 결측값 :
 650_dst    0
660_dst    0
670_dst    0
680_dst    0
690_dst    0
700_dst    0
710_dst    0
720_dst    0
730_dst    0
740_dst    0
750_dst    0
760_dst    0
770_dst    0
780_dst    0
790_dst    0
800_dst    0
810_dst    0
820_dst    0
830_dst    0
840_dst    0
850_dst    0
860_dst    0
870_dst    0
880_dst    0
890_dst    0
900_dst    0
910_dst    0
920_dst    0
930_dst    0
940_dst    0
950_dst    0
960_dst    0
970_dst    0
980_dst    0
990_dst    0
dtype: int64


In [10]:
print("\n___Test dataset")
shownull(test)

X_test_ipt = test.copy()
imputer = IterativeImputer(random_state=2023)
X_test_ipt[missval_name] = imputer.fit_transform(X_test_ipt[missval_name])
print("\n변환 후 결측값 :\n", X_test_ipt[missval_name].isnull().sum())


___Test dataset
데이터셋 차원 확인 
 (10000, 71)
결측값 있는 변수 확인 
 650_dst    2026
660_dst    1960
670_dst    2015
680_dst    2006
690_dst    1985
700_dst    1948
710_dst    2043
720_dst    2014
730_dst    2038
740_dst    1999
750_dst    1965
760_dst    2000
770_dst    1956
780_dst    2020
790_dst    2030
800_dst    1961
810_dst    2034
820_dst    2042
830_dst    2058
840_dst    1947
850_dst    2005
860_dst    2008
870_dst    2063
880_dst    2080
890_dst    1996
900_dst    1976
910_dst    1919
920_dst    1981
930_dst    2000
940_dst    1999
950_dst    1949
960_dst    2020
970_dst    1976
980_dst    2011
990_dst    1970
dtype: int64

변환 후 결측값 :
 650_dst    0
660_dst    0
670_dst    0
680_dst    0
690_dst    0
700_dst    0
710_dst    0
720_dst    0
730_dst    0
740_dst    0
750_dst    0
760_dst    0
770_dst    0
780_dst    0
790_dst    0
800_dst    0
810_dst    0
820_dst    0
830_dst    0
840_dst    0
850_dst    0
860_dst    0
870_dst    0
880_dst    0
890_dst    0
900_dst    0
910_dst    0
920_ds

## Scaler

In [11]:
scaler = StandardScaler()

X_train01 = scaler.fit_transform(X_train_ipt)
X_test = scaler.transform(X_test_ipt)

In [12]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train01, y_train, test_size = 0.2)

# 학습

## XGBoost </p>
의 하이퍼 파라미터를 hyperopt로 튜닝

In [13]:
reg_candidate = [1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1, 5, 10, 100]

SEED = 2023

space={'max_depth': hp.quniform("max_depth", 5, 15, 1),
       'learning_rate': hp.quniform ('learning_rate', 0.01, 0.05, 0.005),
       'subsample': hp.quniform('subsample', 0.6, 1, 0.05),
       'min_child_weight' : hp.quniform('min_child_weight', 1, 10, 1),
       'n_estimators': hp.quniform('n_estimators', 200, 1000, 100)
      }


In [17]:
#학습을 위한 목적 변수 설정
def xgb_hptuning(space) :
    xgb_model = XGBRegressor(n_estimators =int(space['n_estimators']), 
                    max_depth = int(space['max_depth']), 
                    learning_rate = space['learning_rate'],
                    subsample = space['subsample'],
                    min_child_weight = int(space['min_child_weight']),
                    early_stopping_rounds=20,
                    random_state=SEED,
                    eval_metric='rmse'
    )
    evaluation = [(X_train, y_train), (X_valid, y_valid)]
    
    xgb_model.fit(X_train, y_train,
              eval_set=evaluation,
              verbose=0)

    pred = xgb_model.predict(X_valid)
    rmse= RMSE(y_valid, pred)

    # 평가 방식 선정
    return {'loss':rmse, 'status': STATUS_OK, 'model': xgb_model}

In [18]:
# Trials 객체 선언합니다.
trials = Trials()
# best에 최적의 하이퍼 파라미터를 return 받습니다.
best = fmin(fn=xgb_hptuning,
            space=space,
            algo=tpe.suggest,
            max_evals=20, # 최대 반복 횟수를 지정합니다.
            trials=trials)

# 최적화된 결과를 int로 변환해야하는 파라미터는 타입 변환을 수행합니다.
best['max_depth'] = int(best['max_depth'])
best['min_child_weight'] = int(best['min_child_weight'])
best['n_estimators'] = int(best['n_estimators'])
best['random_state'] = SEED

print(best)

100%|██████████| 20/20 [15:24<00:00, 46.21s/trial, best loss: 1.7252715179134075]
{'learning_rate': 0.025, 'max_depth': 8, 'min_child_weight': 9, 'n_estimators': 900, 'subsample': 0.9500000000000001, 'random_state': 2023}


In [19]:
best

{'learning_rate': 0.025,
 'max_depth': 8,
 'min_child_weight': 9,
 'n_estimators': 900,
 'subsample': 0.9500000000000001,
 'random_state': 2023}

In [20]:
xgb = XGBRegressor(**best)
xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)

In [21]:
#결과파일 출력
submit = pd.read_csv("./sample_submission.csv")
submit[list(y_test.columns)] = pred
submit.to_csv('./submit_xgb.csv', index=False)